In [1]:
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(7)

In [2]:
def load_csv():
    with open('../data/subject_grades.csv', mode='r') as csv_file:
        data = list(csv.reader(csv_file, delimiter=',', quotechar='"'))
        data.pop(0)
        
        data = [[float(y) for y in x] for x in data]
        
        train_data = data[:int(len(data)*80/100)]
        test_data = data[int(len(data)*80/100):]
        
        return train_data, test_data

In [3]:
class FinalGradeNN(nn.Module):
    def __init__(self):
        super(FinalGradeNN, self).__init__()
        
        self.fc1 = nn.Linear(2, 1)
        self.output = nn.Linear(1, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.output(x)
        
        return x
    
net = FinalGradeNN()
optimizer = torch.optim.Adam(net.parameters(), lr=0.05)
loss_func = torch.nn.MSELoss()

print(net)

FinalGradeNN(
  (fc1): Linear(in_features=2, out_features=1, bias=True)
  (output): Linear(in_features=1, out_features=1, bias=True)
)


In [4]:
train_data, test_data = load_csv()


y_train = torch.FloatTensor([i[-1:] for i in train_data])
x_train = torch.FloatTensor([i[:2] for i in train_data])

In [7]:
train_data

[[13.0, 13.0, 13.0],
 [3.0, 13.0, 8.0],
 [0.0, 7.0, 3.5],
 [20.0, 6.0, 13.0],
 [18.0, 10.0, 14.0],
 [5.0, 4.0, 4.5],
 [17.0, 19.0, 18.0],
 [17.0, 10.0, 13.5],
 [6.0, 1.0, 3.5],
 [10.0, 18.0, 14.0],
 [2.0, 3.0, 2.5],
 [10.0, 5.0, 7.5],
 [10.0, 6.0, 8.0],
 [11.0, 13.0, 12.0],
 [4.0, 20.0, 12.0],
 [10.0, 3.0, 6.5],
 [3.0, 0.0, 1.5],
 [16.0, 14.0, 15.0],
 [9.0, 8.0, 8.5],
 [4.0, 9.0, 6.5],
 [4.0, 14.0, 9.0],
 [7.0, 1.0, 4.0],
 [18.0, 10.0, 14.0],
 [4.0, 11.0, 7.5],
 [19.0, 20.0, 19.5],
 [17.0, 6.0, 11.5],
 [13.0, 4.0, 8.5],
 [12.0, 5.0, 8.5],
 [1.0, 14.0, 7.5],
 [2.0, 11.0, 6.5],
 [20.0, 12.0, 16.0],
 [8.0, 2.0, 5.0],
 [9.0, 13.0, 11.0],
 [18.0, 18.0, 18.0],
 [10.0, 9.0, 9.5],
 [6.0, 8.0, 7.0],
 [4.0, 6.0, 5.0],
 [15.0, 17.0, 16.0],
 [6.0, 20.0, 13.0],
 [7.0, 4.0, 5.5],
 [0.0, 4.0, 2.0],
 [13.0, 2.0, 7.5],
 [13.0, 7.0, 10.0],
 [17.0, 15.0, 16.0],
 [14.0, 12.0, 13.0],
 [3.0, 4.0, 3.5],
 [12.0, 19.0, 15.5],
 [20.0, 20.0, 20.0],
 [12.0, 2.0, 7.0],
 [6.0, 5.0, 5.5],
 [6.0, 6.0, 6.0],
 [15.0, 1

In [5]:
epochs = 400

for epoch in range(epochs):
    predict = net(x_train)
    
    loss = loss_func(predict, y_train)
    
    if (epoch+1) % 10 == 0:
        print(epoch+1, loss.item())
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

10 51.76478576660156
20 19.98354148864746
30 4.167577743530273
40 0.5395511388778687
50 0.5288690328598022
60 0.41275709867477417
70 0.25596413016319275
80 0.17579646408557892
90 0.13769720494747162
100 0.11633188277482986
110 0.10154905170202255
120 0.08959629386663437
130 0.07904965430498123
140 0.06947740167379379
150 0.060774121433496475
160 0.05289456248283386
170 0.04579635709524155
180 0.03943828493356705
190 0.033779993653297424
200 0.028778765350580215
210 0.024387888610363007
220 0.02055811882019043
230 0.01723889447748661
240 0.01438024640083313
250 0.011933399364352226
260 0.009851817041635513
270 0.008091493509709835
280 0.006611772812902927
290 0.005375196225941181
300 0.004347680602222681
310 0.0034989467822015285
320 0.0028017673175781965
330 0.002232256578281522
340 0.0017697045113891363
350 0.0013960328651592135
360 0.0010958091588690877
370 0.0008559464477002621
380 0.0006652697920799255
390 0.0005145602626726031
400 0.000396052811993286


In [6]:
i = 1
correct_predict = 0

for entry in test_data:
    expected = entry[-1:][0]
    predicted = round(net.forward(torch.FloatTensor(entry[:2])).item(), 1)
    
    if expected == predicted:
        correct_predict += 1
    
    print(str(i) + '. Expected value: ' + str(expected) + 
          ' - Predicted value: ' + str(predicted) +
          (' NOT EQUAL' if expected != predicted else ''))
    i += 1
    
print('Correct predictions: ' + str(correct_predict) +
      ' - Incorrect: ' + str(len(test_data) - correct_predict))

1. Expected value: 6.0 - Predicted value: 6.0
2. Expected value: 11.0 - Predicted value: 11.0
3. Expected value: 9.0 - Predicted value: 9.0
4. Expected value: 12.0 - Predicted value: 12.0
5. Expected value: 12.0 - Predicted value: 12.0
6. Expected value: 5.0 - Predicted value: 5.0
7. Expected value: 7.5 - Predicted value: 7.5
8. Expected value: 12.0 - Predicted value: 12.0
9. Expected value: 9.5 - Predicted value: 9.5
10. Expected value: 13.0 - Predicted value: 13.0
11. Expected value: 12.5 - Predicted value: 12.5
12. Expected value: 7.0 - Predicted value: 7.0
13. Expected value: 2.0 - Predicted value: 2.0
14. Expected value: 10.0 - Predicted value: 10.0
15. Expected value: 4.0 - Predicted value: 4.0
16. Expected value: 9.0 - Predicted value: 9.0
17. Expected value: 5.5 - Predicted value: 5.5
18. Expected value: 0.5 - Predicted value: 0.5
19. Expected value: 3.5 - Predicted value: 3.5
20. Expected value: 4.5 - Predicted value: 4.5
21. Expected value: 8.0 - Predicted value: 8.0
22. Expe